## **使用 Python Madmom 包，Adapter EIM 插件与 Scratch 流动生成频谱图（spectrogram）**

**1. EIM 自启动 Scratch 版**

EIM 相比 Linda，可以更高频率地向 Scratch 发送数据，保证 Scratch 画图与 Python 数据分析的同步性，即实时性。

In [ ]:
import madmom
import numpy as np
import webbrowser
import time
from codelab_adapter_client import AdapterNode


# EIM 初始化
class MyNode(AdapterNode):
    NODE_ID = "eim/spectrogram"

    def __init__(self):
        super().__init__()
        self.is_ready = False
    
    def send_data(self, content):
        message = self.message_template()
        message["payload"]["content"] = content
        self.publish(message)
        
    def extension_message_handle(self, topic, payload):
        self.logger.info(f'the message payload from scratch: {payload}')
        content = payload["content"]
        if content == "ready":
            self.is_ready = True  

node = MyNode()
node.receive_loop_as_thread()
time.sleep(0.1)



# 在新的浏览器窗口自动打开对应 Scratch 项目，使用 EIM 发送消息使 Scratch 项目待命，同时等待项目加载成功后的返回消息
webbrowser.open('https://create.codelab.club/projects/9942/editor/', new=1)

print("Waiting for Scratch.")

while not node.is_ready:  
    node.send_data("go")
    time.sleep(0.5)

print("scratch is ready")     
    


# 使用 Madmom 实时抓取音频流数据，做 sftf 并生成 sepctrogram
kwargs = dict(
    sample_rate=44100,                        # 采样率
    num_channels=1,                           # 单声道
    frame_size=2048,                          # 以 2048 sample 为一个 frame 做 stft 分析，同时也是窗口大小
    hop_size=441,                             # 间隔 441 sample 有重合的取 frame 做 stft 分析
    filterbank=madmom.audio.filters.MelFilterbank,          # 对频率做 MelFilter
    num_bands=12                              # 分为 12 个频段
    
)


stream=madmom.audio.signal.Stream(**kwargs)

spec_list = [i for i in range(10)]            # 以 10 个时间点的结果为单位向 Scratch 发送列表数据

n = 0

try:
    for frames in stream:  
        fs = madmom.audio.signal.FramedSignal(frames,**kwargs)   # 将以 sample 为单位的信号转为以 frame 为单位的信号
        stft = madmom.audio.stft.STFT(fs)                        # 做 stft 分析
        spec = madmom.audio.spectrogram.LogarithmicFilteredSpectrogram(stft, **kwargs)     # 对频率做 MelFilter 同时对强度值做对数转换
        i = n%2
        if i == 0:                                               # 累积两个 frame 的结果发送给 Scratch
            for j in range(len(spec)):
                spec_list[i*5+j]=np.ndarray.tolist(spec[j])
            n = n + 1
        elif i == 1:
            for j in range(len(spec)):
                spec_list[i*5+j]=np.ndarray.tolist(spec[j])
            node.send_data(spec_list)
            n = n + 1   
            
            

except KeyboardInterrupt:
    stream.close()
    print('interrupt by user')


In [ ]:
import madmom
import numpy as np
import webbrowser
import time
from codelab_adapter_client import AdapterNode


# EIM 初始化
class MyNode(AdapterNode):
    NODE_ID = "eim/spectrogram_m"

    def __init__(self):
        super().__init__()
    
    def send_data(self, content):
        message = self.message_template()
        message["payload"]["content"] = content
        self.publish(message)


node = MyNode()
node.receive_loop_as_thread()
time.sleep(0.1)

   

# 使用 Madmom 实时抓取音频流数据，做 sftf 并生成 sepctrogram
kwargs = dict(
    sample_rate=44100,                        # 采样率
    num_channels=1,                           # 单声道
    frame_size=2048,                          # 以 2048 sample 为一个 frame 做 stft 分析，同时也是窗口大小
    hop_size=441,                             # 间隔 441 sample 有重合的取 frame 做 stft 分析
    filterbank=madmom.audio.filters.MelFilterbank,          # 对频率做 MelFilter
    num_bands=12                              # 分为 12 个频段
    
)


stream=madmom.audio.signal.Stream(**kwargs)

spec_list = [i for i in range(10)]            # 以 10 个时间点的结果为单位向 Scratch 发送列表数据

n = 0

try:
    for frames in stream:  
        fs = madmom.audio.signal.FramedSignal(frames,**kwargs)   # 将以 sample 为单位的信号转为以 frame 为单位的信号
        stft = madmom.audio.stft.STFT(fs)                        # 做 stft 分析
        spec = madmom.audio.spectrogram.LogarithmicFilteredSpectrogram(stft, **kwargs)     # 对频率做 MelFilter 同时对强度值做对数转换
        i = n%2
        if i == 0:                                               # 累积两个 frame 的结果发送给 Scratch
            for j in range(len(spec)):
                spec_list[i*5+j]=np.ndarray.tolist(spec[j])
            n = n + 1
        elif i == 1:
            for j in range(len(spec)):
                spec_list[i*5+j]=np.ndarray.tolist(spec[j])
            node.send_data(spec_list)
            n = n + 1   
            
            

except KeyboardInterrupt:
    stream.close()
    print('interrupt by user')


</br>

**2. Linda 滞后版**

In [ ]:
import madmom
import numpy as np
from madmom.audio import signal, hpss, spectrogram
import time
from codelab_adapter_client import AdapterNode



class MyNode(AdapterNode):
    NODE_ID = "linda/test"

    def __init__(self):
        super().__init__()
        
node = MyNode()
node.receive_loop_as_thread()


kwargs = dict(
    sample_rate=44100,
    num_channels=1,
    frame_size=2048,
    hop_size=441,
    filterbank=madmom.audio.filters.MelFilterbank,
    num_bands=12
    
)
stream=signal.Stream(**kwargs)


spec_list = [i for i in range(10)]

n = 0

try:
    for frames in stream:  
        lindaout = ["LindaOut",0]
        fs = madmom.audio.signal.FramedSignal(frames,**kwargs)
        stft = madmom.audio.stft.STFT(fs)
        spec = madmom.audio.spectrogram.LogarithmicFilteredSpectrogram(stft, **kwargs)
        i = n%2
        if i == 0:
            for j in range(len(spec)):
                spec_list[i*5+j]=np.ndarray.tolist(spec[j])
            n = n + 1
        elif i == 1:
            for j in range(len(spec)):
                spec_list[i*5+j]=np.ndarray.tolist(spec[j])
            lindaout[1]=spec_list
            #print(lindaout)
            node.linda_out(lindaout)
            n = n + 1   
            
            

except KeyboardInterrupt:
    print('interrupt by user')
